# A Neural Network Based Approach to Medical Image Recognition
### Jack Savage, Kyle Burack, Tyler Seppala, Josh Michelberg

---
**Introduction**:

Many communities around the world have little to no access to modern health care. Within our own country, there are two major groups with limited access: those without money for health-care, and those who are far from their nearest healthcare provider. In both circumstances listed, artificial intelligence provides a solution. In poverty-stricken areas, human doctors are costly while an open-source software-based diagnosis system would be cheap. In rural areas without access to experienced or well-trained doctors, medical technicians would be able to perform at a similar level.

![alt text](doctor_shortages.jpg "Doctor Shortages")
*Relevent graphic showcasing shortages of doctors within the United States*

_Further information on the 'HPSA score' metric used in the image can be found [here](https://bhw.hrsa.gov/shortage-designation/hpsas)_

## Background

Many different approaches to automated computer diagnosis have been attempted as aids in decision-making in many fields of medicine since the 1950s (Doi K. et al. 2007). With automated computer diagnosis, the performance level of the computer output is required to be very high. For example, if the sensitivity for detection of lesions by computer would be lower than the average sensitivity of physicians, it would be difficult to justify the use of automated computer diagnosis.

By using B.B. Mandelbrot's fractal theory (Barnsley et al. 2012) from 1982, it was discovered that fractal dimension could be obtained in medical images by the concept of fractional Brownian motion. Two applications are found: (1) classification; (2) edge enhancement and detection.
    
A neural network was used for medical image classification in 3 categories: radiology, cardiology, and gastroenterology (Tajbakhsh et al. 2016). The results showed that a pre-trained CNN with a bit of fine-tuning performed a lot better than a deep CNN trained from scratch. Another study saw a CNN trained with 757 disease clauses and was expected to be able to classify skin cancer, classify the type of skin cancer (non-neoplastic, malignant, and benign), and the type of that cancer (Esteva et al. 2017). The developed and fine tuned CNN was run through an ImageNet dataset of about 1.28 million images and had pretty good results.Overall, this is a field with potential that is being heavily researched.

## Past Work

We began this project last semester with no knowledge of neural networks. ~75% of our work last semester was in researching our topic while ~25% was becoming familiar with the high-level API [Keras](https://keras.io/). This semester, we've been able to focus completely on model development and assessment and have created 2 unsuccessful models and 1 moderately successful model. 

Our initial model was of the Xception architecture (Chollet 2017) and was trained on the untouched dataset divided into subclasses. Xception is an adjusted version of Google's publicly released Inception model (Szegedy et al. 2015). Both architectures contain specialized layers where multiple convolutional operations are performed on the same tensor. Inception modules concatonate the outputs of these operations while Xception maps the all outputs directly to a tensor of the same x and y dimensions. 


<img src="inception_module.JPG" alt="inception_module" style="width: 400px;"/>
<img src="xception_module.JPEG" alt="xception_module" style="width: 400px;"/>

*Comparing inception and xception module architecture*


We chose Xception because it's been reliably proven to be more accurate than the other architecture while also being relatively quick to train ([compared to the other architectures available natively within keras](https://keras.io/applications/)). This model proved to be a failure due to reasons discussed in the **Dataset** section. This model was extremely biased towards classifying images as being heathy. 

As the use of single stage algorithms wasn’t proving successful, we decided to try a two stage object detection network called RetinaNet. The backbone of this model is an algorithm known as a feature pyramid network (FPN). The feature pyramid takes an input image and breaks it down to a range of different resolutions, forming a feature map out of them (Lin et al. 2017). 
The feature map at each resolution, or layer, is then fed in succession through two fully convolutional neural networks (FCN’s). The first stage of object detection, carried out by the first FCN,  is object classification. The algorithm breaks the image from the given FPN level into candidate regions, enclosing each one with an anchor box.

<img src="object_detection_example.JPG" alt="xception_module" style="width: 400px;"/>

These candidate regions are parts of the image that the network, based on its training, suspect contain pneumonia. This new image, which contains the anchor boxes, is then fed into the second FCN for the second stage: box regression. This FCN takes a closer, more precise look at the enclosed regions and attempts to resize the anchor boxes to closely fit determined cases of pneumonia. If it is determined that there is no pneumonia within the anchor box, the box will be deleted entirely. The output of each layer is then compared, and the information is accumulated to produce one output image. Ideally, this image will be blank if healthy, or have anchor boxes tightly fit around any cases of pneumonia (Lin et al. 2017).

Our hope in this new approach was that we could improve training speed and classification accuracy by avoiding class imbalance. Class imbalance in this case describes a situation where the classes the network is trying to identify are scarce within each training image, and there is a lot of empty space void of useful information (Lin et al. 2017). We found in our previous attempts that using one network to closely analyze entire images of lungs wasted a lot of training time, as many pneumonia cases exist only in scarce regions. Close analysis of the vast, healthy parts of the lungs is not necessary. We believed that RetinaNet would solve this problem, as it throws away most of the unnecessary information in the first stage. 

<img src="pneumonia_scan_example2.png" alt="scan_example" style="width: 400px;"/>

Because we are fairly new to programming neural networks, and RetinaNet is an existing model that has been successful on other datasets, we decided to utilize a premade model and alter the subnetworks. Here is the repository from which we got the initial code: https://github.com/fizyr/keras-retinanet. We changed the image classification subnetwork to a preset model called ResNet50, which we hoped would better suit our type of training data. Unfortunately, the model was not successful in identifying pneumonia, returning an accuracy below 0.01. On close analysis of our model, we found that the issue was is the classification stage. When evaluated on testing data, anchor boxes were never produced at all, meaning that the classification process was not detailed enough to identify any candidate regions, and the regression network never received any input. 


Unfortunately, our team lost the hard drive containing training and validation metrics for our first two models so will not be able to discuss these models' results in more detail.

## This Semester's Work

**Dataset:**

This semester we've decided to train our model on a Kaggle dataset found [here](https://www.kaggle.com/c/rsna-pneumonia-detection-challenge). We decided to use this dataset primarily because of its size. Medical imaging is difficult to publicly release so large open datasets are rare. The Kaggle Pneumonia dataset contained 25,684 images, which we divided into two classes: "no penumonia" and "yes pneumonia" (unfortunately we're not the most creative). There was a large class imbalance, with 16,720 cases without pneumonia and 8,964 cases with pneumonia. 

With our first two models, we failed to address the class imbalance. This failure presented itself in the results of our initial Xception model. As the model was trained on primarily on images without pneumonia, it had a strong bias to labeling new examples as benign (Buda et al, 2018). This dramatically effected its accuracy on our test dataset and led to the model being useless for a real-world use case. In our second model, the region proposal subnetwork in RetinaNet failed to produce any regions. For our final model, we oversampled the minority class so that the model would train on balanced classes, which improved accuracy.

In addition to the class imbalance, we handled data preprocessing poorly for our first two models. In converting the images to a model readable data type, we downsized the original image from 1000x1000 pixels to the model's default 299x299. In downsampling these images, a lot of the information contained in the image is lost. We originally decided on trying 299x299 due to memory constraints, but were able to access more powerful computing to train our last two models and later used the default image size.

**Our network**

Our network trained through 2.5 iterations of the whole training dataset, training on ~87,500 images over the course of the session. Due to memory constraints when using the full sized images, we had to use a rather low batch size of 3. Below are images of the tensorboard graphs documenting our model's training accuracy and loss by epoch. A model is usually finished training once its loss and accuracy graphs flatten. This indicates that the network is struggling to further fit itself to the data. Pushing on beyond this point would lead to overfitting, as the model molds itself exactly to the data rather than making guesses through generalization.

#### Xception training accuracy
<img src="training_accuracy_xception_v2.PNG" alt="inception_module" style="width: 700px;"/>

#### Xception training loss
<img src="training_loss_xception_v2.PNG" alt="inception_module" style="width: 700px;"/>

**Results**:    
From the graphs above, we can see that our network accuracy on the training data is around .740. This means that the model correctly classified 74% of the training images throughout the training session. Not an awful metric considering the obscurity of the disease in the images, but one that must be improved before even thinking of pushing it as a product.

## Evaluating model on Test Data

The code below evaluates our final model on a completely unseen test set of images evenly split between "yes pneumonia" and "no pneumonia". 

In [1]:
import keras
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# loading model file into keras model object
xception_final = load_model('xception15.hdf5')

In [5]:
data_dir = r'C:\Users\jecks\Documents\School\GCI\gci_data\test' # change this to test data folder

# keras data generators vastly simplify the process of handling 
# labels and conversion to numpy arrays for images
test_datagen = ImageDataGenerator()
test_datagen = test_datagen.flow_from_directory(data_dir,
                                                batch_size = 1,
                                                target_size=(1024,1024),
                                                class_mode='categorical')

Found 208 images belonging to 2 classes.


In [7]:
# test_history contains model metrics on test set
test_history = xception_final.evaluate_generator(test_datagen, verbose=1)

208/208 [==============================] - 2609s 13s/step


In [11]:
print(test_history)
print(xception_final.metrics_names)

[0.6660128929114972, 0.6394230769230769]
['loss', 'acc']


The model's accuracy on test data is 10% below that on training data. This is a somewhat remarkeable change, showing our model is only 14% better than flipping a coin. Major improvements are required moving forward if we wanted to put this model into production. 

**Improvements for Next Semester:**

While this semester's results aren't ideal, we've built a robust codebase to build upon. In less than a year, our group has gone from not knowing anything about neural networks to developing and evaluating multiple models rapidly. In the spring, we have many ways improve our model's accuracy. 

First, we could train more CNNs on our dataset and use ensemble them with our existing model (output is average of each networks output). This is a technique thats been used to great effect elsewhere (Faußer S, Schwenker F 2013). 

Second, we can implement new algorithms discovered in literature if traditional methods fail. We could augment our dataset by randomly altering training images, effectively creating more data for our network to learn from. 

Finally, we could attempt to improve our existing model by tweaking hyperparameters such as number of epochs and steps per epoch which will give the model more training iterations over the whole dataset. This final option likely leads to overfitting, as we can see that our loss and accuracy have already sufficiently flattened out. If the slope was more drastic in the final epochs, this would suggest that our model's loss hasn't hit a local minimum and more training is required. 

## Conclusion

In conclusion, we've made progress this semester by training our first few models but will need to make many improvements moving forwards. Getting our hands dirty with keras and the models allowed our group members to more tangibly understand the concepts discussed last semester. In the future, machines will be able to practice medicine with the same ability as human doctors. This project is an early step forwards. 

## Resources

Chollet F. 2017. Xception: Deep Learning with Depthwise Separable Convolutions. 2017 IEEE Conference on Computer Vision and Pattern Recognition (CVPR).

Szegedy C, Liu W, Jia Y, Sermanet P, Reed S, Anguelov D, Erhan D, Vanhoucke V, Rabinovich A. 2015. Going deeper with convolutions. 2015 IEEE Conference on Computer Vision and Pattern Recognition (CVPR).

Tajbakhsh, N., J. Y. Shin, S. R. Gurudu, R. T. Hurst, C. B. Kendall, M. B. Gotway, and J. Liang. 2016. “Convolutional Neural Networks for Medical Image Analysis: Full Training or Fine Tuning?” IEEE Transactions on Medical Imaging 35 (5): 1299–1312. https://doi.org/10.1109/TMI.2016.2535302.

Esteva, Andre, Brett Kuprel, Roberto A. Novoa, Justin Ko, Susan M. Swetter, Helen M. Blau, and Sebastian Thrun. 2017. “Dermatologist-Level Classification of Skin Cancer with Deep Neural Networks.” Nature 542 (7639): 115–18. https://doi.org/10.1038/nature21056.

Doi K. 2007. Computer-aided diagnosis in medical imaging: Historical review, current status and future potential. Computerized Medical Imaging and Graphics 31:1–211.

Barnsley MF, Devaney RL, Mandelbrot BB, Voss RF. 2012. Science of fractal images. Peitgen HO, Saupe D, editors. Place of publication not identified: Springer-Verlag New York.

Buda M, Maki A, Mazurowski MA. 2018. A systematic study of the class imbalance problem in convolutional neural networks. Neural Networks 106:249–259.

Lin T-Y, Goyal P, Girshick R, He K, Dollar P. 2017. Focal Loss for Dense Object Detection. 2017 IEEE International Conference on Computer Vision (ICCV).

Lin, Tsung-Yi, Piotr Dollar, Ross Girshick, Kaiming He, Bharath Hariharan, and Serge Belongie. 2017. “Feature Pyramid Networks for Object Detection.” 2017 IEEE Conference on Computer Vision and Pattern Recognition (CVPR). doi:10.1109/cvpr.2017.106.	

Faußer S, Schwenker F. 2013. Neural Network Ensembles in Reinforcement Learning. Neural Processing Letters 41:55–69.

Courtney DM, Self WH, Mcnaughton CD, Wunderink RG, Kline JA. 2013. High discordance of chest x-ray and computed tomography for detection of pulmonary opacities in ED patients: implications for diagnosing pneumonia. The American Journal of Emergency Medicine 31:401–405.